# Gym Environment
In order for the agent to interact with the environment, we need an environment for our agent to exist in. This will simply adapt visuals present in the game into actions/observations to be made.

## Event Detector
The only events we need to detect is when the player dies or completes a level.

In [1]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
# manually created crop areas
# locations are at...
# complete level tag
#   plt.imshow(frame[110:150, 120:520])
#   120, 110, 400, 40
# retry button
#   plt.imshow(frame[375:425, 160:215])
#   160, 375, 55, 50

In [67]:
cap = cv2.VideoCapture(1)  # Use your screen capture setup
ret, frame = cap.read()
cap.release()

In [68]:
class EventDetectorWithROI:
    def __init__(self, completion_template_path, retry_template_path, completion_roi, retry_roi,):
        # templates
        self.completion_template = cv2.imread(completion_template_path, cv2.IMREAD_GRAYSCALE)
        self.retry_template = cv2.imread(retry_template_path, cv2.IMREAD_GRAYSCALE)

        # roi regions
        self.retry_roi = retry_roi
        self.completion_roi = completion_roi

    def _crop_to_roi(self, frame, roi):
        """
        Crop the frame to the given ROI.
        """
        x, y, w, h = roi
        return frame[y:y+h, x:x+w]

    def detect_event(self, frame):
        """
        Detect if the player has died or completed the level.
        
        returns tring "death", "completion", or None
        """
        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Crop the ROIs
        retry_roi_frame = self._crop_to_roi(gray_frame, self.retry_roi)
        completion_roi_frame = self._crop_to_roi(gray_frame, self.completion_roi)

        # Detect completion (Completion screen)
        completion_res = cv2.matchTemplate(completion_roi_frame, self.completion_template, cv2.TM_CCOEFF_NORMED)
        completion_threshold = 0.8
        if (completion_res >= completion_threshold).any():
            return "completion"
        
        # Detect death (Retry button), also after completion
        retry_res = cv2.matchTemplate(retry_roi_frame, self.retry_template, cv2.TM_CCOEFF_NORMED)
        retry_threshold = 0.8
        if (retry_res >= retry_threshold).any():
            return "death"

        # Detect completion (Completion screen)
        completion_res = cv2.matchTemplate(completion_roi_frame, self.completion_template, cv2.TM_CCOEFF_NORMED)
        completion_threshold = 0.8
        if (completion_res >= completion_threshold).any():
            return "completion"

        return None


In [69]:
# In (x, y, width, height)
completion_roi = (120, 110, 400, 40)
retry_roi = (160, 375, 55, 50) 

# Initialize the detector with templates and ROIs
detector = EventDetectorWithROI(
    "./template_img/complete_template.png",
    "./template_img/retry_template.png",
    completion_roi,
    retry_roi
)

# Test the detector
cap = cv2.VideoCapture(1)  # Use your screen capture setup
ret, frame = cap.read()
cap.release()

event = detector.detect_event(frame)
if event == "death":
    print("Player died!")
elif event == "completion":
    print("Level completed!")
else:
    print("No event detected.")

Player died!


## Input Controller
First thing is to create a framework to interact with the GD game.